 # Stack Overflow Developer Survey 2020 - private

In [1]:
import pandas
import numpy


  Data downloaded from Stack Overflow

In [2]:
source = pandas.read_csv('survey_results_public.csv')
source.head()


,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,...,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
0,1,I am a developer by profession,Yes,NaN,13,Monthly,NaN,NaN,Germany,European Euro,...,Neither easy nor difficult,Appropriate in length,No,"Computer science, computer engineering, or sof...",ASP.NET Core,ASP.NET;ASP.NET Core,Just as welcome now as I felt last year,50.0,36,27
1,2,I am a developer by profession,No,NaN,19,NaN,NaN,NaN,United Kingdom,Pound sterling,...,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat more welcome now than last year,NaN,7,4
2,3,I code primarily as a hobby,Yes,NaN,15,NaN,NaN,NaN,Russian Federation,NaN,...,Neither easy nor difficult,Appropriate in length,NaN,NaN,NaN,NaN,Somewhat more welcome now than last year,NaN,4,NaN
3,4,I am a developer by profession,Yes,25.0,18,NaN,NaN,NaN,Albania,Albanian lek,...,NaN,NaN,No,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat less welcome now than last year,40.0,7,4
4,5,"I used to be a developer by profession, but no...",Yes,31.0,16,NaN,NaN,NaN,United States,NaN,...,Easy,Too short,No,"Computer science, computer engineering, or sof...",Django;Ruby on Rails,Ruby on Rails,Just as welcome now as I felt last year,NaN,15,8


  ## Liked technologies
  ### Step by step
  #### Languages

In [3]:
language_likes_source = source[['LanguageWorkedWith','LanguageDesireNextYear']]
language_likes_source.head()


,LanguageWorkedWith,LanguageDesireNextYear
0,C#;HTML/CSS;JavaScript,C#;HTML/CSS;JavaScript
1,JavaScript;Swift,Python;Swift
2,Objective-C;Python;Swift,Objective-C;Python;Swift
3,NaN,NaN
4,HTML/CSS;Ruby;SQL,Java;Ruby;Scala


In [4]:
complete_rows_only = language_likes_source.dropna()
complete_rows_only.head()


,LanguageWorkedWith,LanguageDesireNextYear
0,C#;HTML/CSS;JavaScript,C#;HTML/CSS;JavaScript
1,JavaScript;Swift,Python;Swift
2,Objective-C;Python;Swift,Objective-C;Python;Swift
4,HTML/CSS;Ruby;SQL,Java;Ruby;Scala
5,HTML/CSS;Java;JavaScript,HTML/CSS;Java;JavaScript


  split the lists in both columns

In [5]:
def split_all_lists(dataframe_):
    splited = pandas.DataFrame()
    for column in dataframe_:
        splited[column] = dataframe_[column].str.split(';')
    return splited


In [6]:
languages_splited = split_all_lists(complete_rows_only)
languages_splited.head()


,LanguageWorkedWith,LanguageDesireNextYear
0,"[C#, HTML/CSS, JavaScript]","[C#, HTML/CSS, JavaScript]"
1,"[JavaScript, Swift]","[Python, Swift]"
2,"[Objective-C, Python, Swift]","[Objective-C, Python, Swift]"
4,"[HTML/CSS, Ruby, SQL]","[Java, Ruby, Scala]"
5,"[HTML/CSS, Java, JavaScript]","[HTML/CSS, Java, JavaScript]"


  make a row for every item in first column

In [7]:
def explode_first_column(dataframe_):
    return dataframe_.explode(dataframe_.columns[0])


In [8]:
exploded = explode_first_column(languages_splited)
exploded.head()


,LanguageWorkedWith,LanguageDesireNextYear
0,C#,"[C#, HTML/CSS, JavaScript]"
0,HTML/CSS,"[C#, HTML/CSS, JavaScript]"
0,JavaScript,"[C#, HTML/CSS, JavaScript]"
1,JavaScript,"[Python, Swift]"
1,Swift,"[Python, Swift]"


  function that compares technology worked in and technologies desired in a row

In [9]:
def is_liked(row):
    return row[0] in row[1]


 function to apply is_liked on dataframe

In [10]:
def add_likes(dataframe_):
    dataframe_['liked'] = dataframe_.apply(is_liked, axis=1)
    return dataframe_


  create new column with True if responder wants to continue developing in technology, else False

In [11]:
with_likes = add_likes(exploded)
with_likes.head()


,LanguageWorkedWith,LanguageDesireNextYear,liked
0,C#,"[C#, HTML/CSS, JavaScript]",True
0,HTML/CSS,"[C#, HTML/CSS, JavaScript]",True
0,JavaScript,"[C#, HTML/CSS, JavaScript]",True
1,JavaScript,"[Python, Swift]",False
1,Swift,"[Python, Swift]",True


In [15]:
def aggregate_likes(dataframe_):
    return dataframe_.groupby(dataframe_.columns[0]).agg({'liked':'mean'})


In [14]:
languages_unsorted = aggregate_likes(exploded)
languages_unsorted.head()


,liked
LanguageWorkedWith,
Assembly,0.317697
Bash/Shell/PowerShell,0.567431
C,0.355820
C#,0.639605
C++,0.468634


In [16]:
languages_sorted = languages_unsorted.sort_values('liked', ascending=False)
languages_sorted


,liked
LanguageWorkedWith,
Rust,0.896907
TypeScript,0.713733
Python,0.708647
Julia,0.700651
Kotlin,0.672006
Dart,0.668712
Go,0.659449
Swift,0.641587
C#,0.639605


  ### In one step

In [17]:
def likes(root):
    workedwith = ''.join([root, 'WorkedWith'])
    desirenextyear = ''.join([root, 'DesireNextYear'])
    dataframe_part = source[[workedwith, desirenextyear]].dropna()
    splited = split_all_lists(dataframe_part)
    exploded = explode_first_column(splited)
    with_likes_ = add_likes(exploded)
    aggregated = aggregate_likes(with_likes_)
    return aggregated.sort_values('liked', ascending=False)


In [18]:
language_likes = likes('Language')
language_likes


,liked
LanguageWorkedWith,
Rust,0.896907
TypeScript,0.713733
Python,0.708647
Julia,0.700651
Kotlin,0.672006
Dart,0.668712
Go,0.659449
Swift,0.641587
C#,0.639605


In [19]:
liked_databases = likes('Database')
liked_databases


,liked
DatabaseWorkedWith,
Redis,0.750623
PostgreSQL,0.735508
Elasticsearch,0.672999
MongoDB,0.648668
Firebase,0.629767
Microsoft SQL Server,0.591187
DynamoDB,0.587533
MariaDB,0.582071
SQLite,0.576664


In [21]:
platform_likes = likes('Platform')
platform_likes

,liked
PlatformWorkedWith,
Linux,0.831118
Docker,0.796395
Kubernetes,0.784401
AWS,0.724538
Raspberry Pi,0.721609
MacOS,0.710376
iOS,0.684309
Microsoft Azure,0.677950
Google Cloud Platform,0.672062


In [20]:
webframe_likes = likes('Webframe')
webframe_likes

,liked
WebframeWorkedWith,
ASP.NET Core,0.791055
React.js,0.790325
Vue.js,0.744114
Express,0.687477
Gatsby,0.670993
Spring,0.665946
Django,0.649746
Flask,0.643984
Angular,0.610726


In [22]:
misctech_likes = likes('MiscTech')
misctech_likes


,liked
MiscTechWorkedWith,
.NET Core,0.790930
Torch/PyTorch,0.790893
Teraform,0.781635
Flutter,0.779038
Node.js,0.758955
Pandas,0.756546
TensorFlow,0.742773
Keras,0.738022
Ansible,0.677913


In [23]:
collab_tool_likes = likes('NEWCollabTools')
collab_tool_likes


,liked
NEWCollabToolsWorkedWith,
Github,0.882721
Gitlab,0.761262
Slack,0.747640
Stack Overflow for Teams,0.738284
"Google Suite (Docs, Meet, etc)",0.735846
Microsoft Azure,0.712099
Jira,0.605373
Trello,0.576335
Confluence,0.536304


 ## Salaries
  limit salaries to meaningful values

In [ ]:
MINIMAL_WAGE = 13000
MAXIMAL_WAGE = 1999999


  ... to be continued ...

In [ ]:
'LanguageWorkedWith'
'LanguageDesireNextYear'
'DatabaseWorkedWith'
'DatabaseDesireNextYear'
'PlatformWorkedWith'
'PlatformDesireNextYear'
'WebframeWorkedWith'
'WebframeDesireNextYear'
'MiscTechWorkedWith'
'MiscTechDesireNextYear'
'NEWCollabToolsWorkedWith'
'NEWCollabToolsDesireNextYear'
'DevType'
'ConvertedComp'
'YearsCode'
